In [3]:
### Prerna Singh (edited by John D'Uva)

# Isolate cell regions and non-cell regions (using centers of mass) for 3D sliced Clarity images
# Use a brain-wide illumination correction for intensity
# Extract 10x the non-cell data than cell data by selecting points within 20 pixels x y and z of cells

In [1]:
import numpy as np
import os
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import random
import csv
import pandas as pd
import cv2

from cloudvolume import CloudVolume
from tifffile import imsave

In [31]:
# brain section ID
section = '6R'
# create new folder where slices will live
os.makedirs(section)
# date depends when each brain section was saved
date = '180703'

In [32]:
# download a whole image stack into a numpy array from the cloud
vol = CloudVolume('https://dlab-colm.neurodata.io/brian-dev/'+date+'_ENTANGLaGFP-'+section+'/Ch1_470_bias_corrected/')
image = vol[:,:,:] 

Downloading: 100%|██████████| 2175/2175 [14:58<00:00,  5.82it/s]

In [33]:
# save each of 'i' z-planes locally (from the tiff stack)
for i in range(0,image.shape[2]):
    if i <10:
        name = "brian-dev_"+date+"_ENTAGLaGP-"+section+"_Ch0_470_z0-2160_y0-2560_z000%i" %i
    elif i <100:
        name = "brian-dev_"+date+"_ENTAGLaGP-"+section+"_Ch0_470_z0-2160_y0-2560_z00%i" %i
    elif i <1000:
        name = "brian-dev_"+date+"_ENTAGLaGP-"+section+"_Ch0_470_z0-2160_y0-2560_z0%i" %i
    else:
        name = "brian-dev_"+date+"_ENTAGLaGP-"+section+"_Ch0_470_z0-2160_y0-2560_z%i" %i
    imsave(section+"/%s.tiff" %name, image[:,:,i])

In [34]:
# create directory
file_path_img = os.path.join(section)
list = os.listdir(file_path_img)

In [35]:
#preprocess the csv with file labels
fileLabelCSV = pd.read_csv('/Users/johnduva/Desktop/2020/csvs/E'+section+'-training.csv')

#make ordered lists of each coordinate dimension
xfileLabelCSV=(fileLabelCSV['Center of Mass (Geometry): X (px)']).to_list()
yfileLabelCSV=(fileLabelCSV['Center of Mass (Geometry): Y (px)']).to_list()
zfileLabelCSV=(fileLabelCSV['Center of Mass (Geometry): Z (px)']).to_list()
      
ROI = np.zeros((len(xfileLabelCSV),12,12,12))

#create a list of ranges +-6 from the cell coordinate that are invalid when accumulating non-cell regions
x_invalid=[]
y_invalid=[]
z_invalid=[]
for index0 in range(0,len(xfileLabelCSV)):
    voxel = np.zeros((12,12,12))
    #current coordinates
    xcoord = int(xfileLabelCSV[index0])
    ycoord = int(yfileLabelCSV[index0])
    zcoord = int(zfileLabelCSV[index0])
    
    #for the ncrs
    x_invalid+=[*range(int(xfileLabelCSV[index0])-6,int(xfileLabelCSV[index0])+6)]
    y_invalid+=[*range(int(yfileLabelCSV[index0])-6,int(yfileLabelCSV[index0])+6)]
    z_invalid+=[*range(int(zfileLabelCSV[index0])-6,int(zfileLabelCSV[index0])+6)]
    
    
    #for every z, append the x&ys
    for index, i in enumerate(range(zcoord-6,zcoord+6)):
        for file in list:
            if str("tif") in str(file):
                if len(str(i))==1:
                    i=str("000"+str(i))
                if len(str(i))==2:
                    i=str("00"+str(i))
                if len(str(i))==3:
                    i=str("0"+str(i))
                if str(i) in str(file).split("_")[7]:
                    scan1 = np.asarray(Image.open(file_path_img + '/'+file))
                    scan = np.zeros(scan1.shape)
                    scan = cv2.normalize(scan1,scan, 0, 255, cv2.NORM_MINMAX)
                    plane = scan[xcoord-6:xcoord+6,ycoord-6:ycoord+6]
                    #plt.imshow(plane)
                    #plt.show()
        voxel[index]=np.asarray(plane)
    ROI[index0]=np.asarray(voxel)

print('Total number of CellRegions: ', len(zfileLabelCSV))

Total number of CellRegions:  316


In [36]:
np.save('ROI'+section+'_bias_corrected.npy', ROI)

In [37]:
#export cell coordinates as .npy
xcoordinate=(fileLabelCSV['Center of Mass (Geometry): X (px)']).to_numpy()
ycoordinate=(fileLabelCSV['Center of Mass (Geometry): Y (px)']).to_numpy()
zcoordinate=(fileLabelCSV['Center of Mass (Geometry): Z (px)']).to_numpy()

In [38]:
#number of non-coding regions we want to isolate, lets say 515, same as number of cell regions
nNCMB = len(zfileLabelCSV)*10
NCR = []

#have 11 different coordinates for 1 non-cell region
for index0 in range(0,len(xfileLabelCSV)):
    #current coordinates
    xcoord = int(xfileLabelCSV[index0])
    ycoord = int(yfileLabelCSV[index0])
    zcoord = int(zfileLabelCSV[index0])
    ncrcell=[]
    while (len(ncrcell)) < 10:
        x_random = random.randint(7,20)
        y_random = random.randint(7,20)
        z_random = random.randint(7,20)
        #proceed =1
        #print("Random x,y,z for Non CMB Region:", x_random,y_random,z_random)

        #for i in x_invalid:
        #    if x_random == i:
        #        #print('x_false')
        #        proceed = 0
        #for i in y_invalid:
        #    if y_random == i:
        #        #print('y_false')
        #        proceed = 0
        #for i in z_invalid:
        #    if z_random == i:
        #        #print('z_false')
        #        proceed = 0
        if len(ncrcell) %8==0:
            #print("proceeding")
            ncrcell.append([xcoord+x_random,ycoord+y_random,zcoord+z_random])
        elif len(ncrcell) %8==1:
            #print("proceeding")
            ncrcell.append([xcoord-x_random,ycoord+y_random,zcoord+z_random])
        elif len(ncrcell) %8==2:
            #print("proceeding")
            ncrcell.append([xcoord+x_random,ycoord-y_random,zcoord+z_random])
        elif len(ncrcell) %8==3:
            #print("proceeding")
            ncrcell.append([xcoord+x_random,ycoord+y_random,zcoord-z_random])
        elif len(ncrcell) %8==4:
            #print("proceeding")
            ncrcell.append([xcoord+x_random,ycoord-y_random,zcoord-z_random])
        elif len(ncrcell) %8==5:
            #print("proceeding")
            ncrcell.append([xcoord-x_random,ycoord+y_random,zcoord-z_random])
        elif len(ncrcell) %8==6:
            #print("proceeding")
            ncrcell.append([xcoord-x_random,ycoord-y_random,zcoord+z_random])
        else:
            ncrcell.append([xcoord-x_random,ycoord-y_random,zcoord-z_random])
    for i in ncrcell:
        NCR.append(i)
    #print(len(NCR))
print("%s non-CMB regions were isolated." %(len(NCR)), "Adding to the master dictionary.")

3160 non-CMB regions were isolated. Adding to the master dictionary.


In [39]:
NCR_total = np.zeros((len(NCR),12,12,12))

for index0 in range(0,len(NCR)):
    voxel = np.zeros((12,12,12))
    
    #current coordinates
    xcoord = int(NCR[index0][0])
    ycoord = int(NCR[index0][1])
    zcoord = int(NCR[index0][2])

    #for every z, append the x&ys
    for index,i in enumerate(range(zcoord-6,zcoord+6)):
        for file in list:
            if str("tif") in str(file):
                if len(str(i))==1:
                    i=str("000"+str(i))
                if len(str(i))==2:
                    i=str("00"+str(i))
                if len(str(i))==3:
                    i=str("0"+str(i))
                if str(i) in str(file).split("_")[7]:
                    scan1 = np.asarray(Image.open(file_path_img + '/'+file))
                    scan = np.zeros(scan1.shape)
                    scan = cv2.normalize(scan1,scan, 0, 255, cv2.NORM_MINMAX)
                    plane = scan[xcoord-6:xcoord+6,ycoord-6:ycoord+6]
        voxel[index] = np.asarray(plane)
    NCR_total[index0]=np.asarray(voxel)

In [40]:
np.save('NCR'+section+'_bias_corrected.npy',np.asarray(NCR_total))

In [41]:
NCR[2]

[249, 784, 657]

In [42]:
image.shape[2]

1389

In [43]:
plt.imshow(scan[NCR[34][0]-30:NCR[34][0]+30,NCR[34][1]-30:NCR[34][1]+30])
plt.show()

In [44]:
plt.imshow(NCR_total[34][5])